In [1]:
# Actually gb can be applied on top of every weak learner (i suppose so)
# So the idea is to implement interface for gb

In [1]:
from utils import split
import linear_models
import importlib

['/Users/krabchuk/tmp/ml_basics/gradient_boosting', '/Users/krabchuk/tmp/ml_basics', '/Users/krabchuk/anaconda3/lib/python310.zip', '/Users/krabchuk/anaconda3/lib/python3.10', '/Users/krabchuk/anaconda3/lib/python3.10/lib-dynload', '', '/Users/krabchuk/anaconda3/lib/python3.10/site-packages', '/Users/krabchuk/anaconda3/lib/python3.10/site-packages/PyQt5_sip-12.11.0-py3.10-macosx-11.1-arm64.egg', '/Users/krabchuk/anaconda3/lib/python3.10/site-packages/aeosa', '/Users/krabchuk/anaconda3/lib/python3.10/site-packages/mpmath-1.2.1-py3.10.egg', '/Users/krabchuk/anaconda3/lib/python3.10/site-packages/pycurl-7.45.1-py3.10-macosx-11.1-arm64.egg']


In [2]:
import numpy as np
from sklearn.metrics import roc_auc_score, mean_squared_error
from sklearn.linear_model import LinearRegression

In [3]:
N = 1000
data = np.random.rand(N, 3)
target = 10 * data[:,0] + 20 * data[:,1] + 30 * data[:,2] + np.random.normal(size=N)

In [4]:
train_data, train_target, test_data, test_target = split(data, target, 0.8)

In [5]:
class LinearModel:
    def __init__(self):
        self.weights = None
        self.train_losses = None

    def __make_iteration(self, X, y, lr, min_loss=None, debug_print=False):
        y_pred = np.dot(X, self.weights)
        if debug_print:
            print(self.weights)
        loss = np.sum((y - y_pred) ** 2) / len(y)
        self.train_losses.append(loss)
        grad = np.dot(y_pred - y, X)
        if min_loss is not None and np.sum(np.abs(grad)) < min_loss:
            return
        self.weights -= lr * grad

    def fit(self, X, y, lr=0.01, iterations=1000, min_loss=1e-9, debug_print=False):
        self.weights = np.random.rand(X.shape[1])
        self.train_losses = []
        for i in range(iterations):
            self.__make_iteration(X, y, lr, min_loss, debug_print)

    def predict(self, X):
        return np.dot(X, self.weights)

In [18]:
importlib.reload(linear_models)
m = linear_models.LinearModel()
m.fit(train_data, train_target, lr=1e-4, iterations=10000, min_loss=1e-9, debug_print=False)
mean_squared_error(test_target, m.predict(test_data))

1.0448220510666841

In [31]:
m_val = LinearRegression()
m_val.fit(train_data, train_target)
mean_squared_error(test_target, m_val.predict(test_data))

0.8962915910317791

In [65]:
from sklearn.tree import DecisionTreeRegressor

In [92]:
models = []
X = train_data
y = train_target
alpha = 1e-2
y_pred = np.full(y.shape, y.mean())
for i in range(100):
    print(f'---iteration {i}---')
    if i != 0:
        y = -(y - y_pred) # mse gradient
    m = DecisionTreeRegressor(max_depth=5)
    m.fit(X, y)
    print(f'weak learner mse = {mean_squared_error(y, m.predict(X))}')
    models.append(m)
    # print(m.predict(X)[:5])
    # print(alpha * m.predict(X)[:5])
    if i != 0:
        y_pred -= alpha * m.predict(X)
    else:
        y_pred = m.predict(X)

    print(train_target[:5])
    print(y_pred[:5])

    test_predict = models[0].predict(test_data)
    for j in range(1, len(models)):
        test_predict -= alpha * models[j].predict(test_data)
    print(f'full model mse {mean_squared_error(test_target, test_predict)}')




---iteration 0---
weak learner mse = 7.736147384579066
[ 8.65391422 29.85484335  9.99338351 22.53599525 24.14363856]
[13.04019055 29.38002075  7.54361902 24.05502309 20.27261772]
full model mse 31.792190603051186
---iteration 1---
weak learner mse = 5.12318609963241
[ 8.65391422 29.85484335  9.99338351 22.53599525 24.14363856]
[13.00984195 29.38855068  7.53771556 24.03355464 20.26671426]
full model mse 31.72223419199049
---iteration 2---
weak learner mse = 7.788637367174377
[ 8.65391422 29.85484335  9.99338351 22.53599525 24.14363856]
[12.87957077 29.09470874  7.46235471 23.7930307  20.06399567]
full model mse 32.75441587505106
---iteration 3---
weak learner mse = 5.051559968083531
[ 8.65391422 29.85484335  9.99338351 22.53599525 24.14363856]
[12.84865212 29.10581443  7.45871988 23.77438403 20.06036084]
full model mse 32.66492310985155
---iteration 4---
weak learner mse = 7.842349408479095
[ 8.65391422 29.85484335  9.99338351 22.53599525 24.14363856]
[12.71849394 28.81189701  7.3834499

In [69]:
m = DecisionTreeRegressor(max_depth=3)
m.fit(X, y)
mean_squared_error(test_target, m.predict(test_data))

42.31018750256522